# Setting

18~49세 전국 성인 남녀 1000명이 조사에 응해주었다. 응답률은 5.4%, 표본오차는 95% 신뢰수준에서 ±3.1%포인트다
미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%, 의향이 없다는 사람은 43.5%였다. 
그런데 여기서 남녀 성별 격차가 나타난다. 미혼 남성은 65.7%가 결혼하고 싶다고 답한다. 
그러나 여성은 54.5%가 결혼할 의향이 없다고 답한다.
(미혼 여성은 45.5%가 결혼하고 싶다고 답한다)

In [1]:
import pymc3 as pm
import arviz
from matplotlib import gridspec
from matplotlib.lines import Line2D
from scipy import stats

In [2]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [3]:
n_all = 1000
n_single = 579 # 미혼남녀
c_level = .95 # 신뢰수준 95%
alpha = 1 - c_level 

In [4]:
def print_interval(low, high):
    #low, high = low*100, high*100
    p = (low+high)/2
    err = high - p
    #print(f'confidence interval: {p:.0f} ± {err:.2f} % [{low:.0f}, {high:.0f}]')
    print(f'confidence interval: {p:.3f} ± {err:.4f}, [{low:.3f}, {high:.3f}]')
    
    
def test_hypothesis(pvalue, level=c_level):
    alpha = 1 - c_level
    if pvalue > alpha:
        s = 'Accept'
    else:
        s = 'Reject'
    print(f'{s} H0 (p-value = {pvalue:.4e})')

# Binomial Testing

## ST01
성인 남녀 1000명이 조사에 응해주었다. 응답률은 5.4%, 표본오차는 95% 신뢰수준에서 ±3.1%포인트다

In [5]:
k = 500 # 어떤 한 질문에 절반이 yes로 대답하는 경우
n = n_all

Confidence interval for a binomial proportion with asymtotic normal approximation

In [6]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
low, high

(0.4690102483847719, 0.5309897516152281)

In [7]:
p = k/n
p-low, high-p

(0.0309897516152281, 0.0309897516152281)

In [8]:
# 표본오차 ±3.1%포인트는 긍정/부정 설문에 50%가 긍정했을때의 오차인 셈
print_interval(low, high)

confidence interval: 0.500 ± 0.0310, [0.469, 0.531]


In [9]:
# 70%가 긍정시 표본오차는 2.8%
p = 0.7
low, high = proportion_confint(int(p*n), n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.700 ± 0.0284, [0.672, 0.728]


## ST02
미혼 응답자 579명 중 결혼 의향이 있다는 사람은 56.5%

In [10]:
n = n_single
p_obs = .565
k = int(n*p_obs) # 결혼의향 있는 사람 수

### CS1 
Confidence interval for a binomial proportion with asymtotic normal approximation

In [11]:
method = 'normal'
#method = 'binom_test' # killed due to small mem size

In [12]:
# 결혼의향 비율에 대한 오차는 ±4%로 설문조사 처음 언급한 표본오차 ±3.1%보다 큼
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.565 ± 0.0404, [0.524, 0.605]


### CS2
binomial test
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [13]:
p = .5
alternative = 'two-sided'

In [14]:
result = stats.binomtest(k, n, p, alternative=alternative)
result

BinomTestResult(k=327, n=579, alternative='two-sided', proportion_estimate=0.5647668393782384, pvalue=0.0020762662419686645)

In [15]:
# reject H0: 결혼의향 p 가정시 p_obj(k/n)일 가능성은 5% 보다 매우 작다
test_hypothesis(result.pvalue)

Reject H0 (p-value = 2.0763e-03)


In [19]:
# proportion_confint과 유사한 결과 (see CS1)
ci = result.proportion_ci(confidence_level=c_level)
print_interval(ci.low, ci.high)

confidence interval: 0.564 ± 0.0412, [0.523, 0.606]


### CS3
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50% 초과

In [20]:
p = .5
alternative = 'greater'

In [21]:
result = stats.binomtest(k, n, p, alternative=alternative)
result

BinomTestResult(k=327, n=579, alternative='greater', proportion_estimate=0.5647668393782384, pvalue=0.0010381331209843323)

In [22]:
# reject H0
test_hypothesis(result.pvalue)

Reject H0 (p-value = 1.0381e-03)


In [23]:
# CS2에 비해 p값은 더 작으나(더 확실하게 H0를 기각) 신뢰구간은 더 넓어짐
# see https://stats.oarc.ucla.edu/other/mult-pkg/faq/general/faq-what-are-the-differences-between-one-tailed-and-two-tailed-tests/
result.proportion_ci(confidence_level=c_level)

ConfidenceInterval(low=0.5298229755863375, high=1.0)

### CS4
Test for proportions based on normal (z) test
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [24]:
p = .5
alternative='two-sided'
#alternative='larger'

In [25]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.6701e-03)


## ST03
미혼 남성은 65.7%가 결혼하고 싶다고 답한다.

In [50]:
n = int(n_single/2) # 미혼남녀 동일비율로 가정
p_obs = .657
k = int(n*p_obs)

### CS1
- H0: 결혼의향 비율은 56.5% (남녀전체 비율)
- Ha: 결혼의향 비율은 56.5%이 아니다

In [51]:
p = .565
alternative = 'two-sided'

In [52]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.4736e-03)


In [53]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.654 ± 0.0548, [0.599, 0.709]


In [54]:
result = stats.binomtest(k, n, p, alternative=alternative)
ci = result.proportion_ci(confidence_level=c_level)
print_interval(ci.low, ci.high)

confidence interval: 0.652 ± 0.0563, [0.596, 0.709]


### CS2
- H0: 결혼의향 비율은 50%
- Ha: 결혼의향 비율은 50%가 아니다

In [32]:
p = .5
alternative = 'two-sided'

In [33]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 3.7396e-08)


## ST04
여성은 54.5%가 결혼할 의향이 없다고 답한다.
(미혼 여성은 45.5%가 결혼하고 싶다고 답한다)

In [34]:
n = int(n_single/2) # 미혼남녀 동일비율로 가정
p_obs = .455
k = int(n*p_obs)

### CS1
- H0: 결혼의향 비율은 56.5% (남녀전체 비율)
- Ha: 결혼의향 비율은 56.5%이 아니다

In [35]:
p = .565
alternative = 'two-sided'

In [36]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 1.3623e-04)


In [37]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.453 ± 0.0574, [0.396, 0.511]


### CS2
결혼의향 없음

In [55]:
p_obs = .545
k = int(n*p_obs)

In [56]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.543 ± 0.0574, [0.486, 0.601]


### CS3
결혼의향 있고, 여성비율 새로 가정

In [62]:
rf = 0.3 # 여성비율 가정
n = int(n_single*rf)
p_obs = .455
k = int(n*p_obs)

In [63]:
p = .565
alternative = 'two-sided'

In [64]:
# reject H0
_, pval = proportions_ztest(k, n, p, alternative=alternative)
test_hypothesis(pval)

Reject H0 (p-value = 2.5531e-03)


In [65]:
method = 'normal'
low, high = proportion_confint(k, n, alpha, method=method)
print_interval(low, high)

confidence interval: 0.451 ± 0.0741, [0.377, 0.525]


In [ ]:
μ_m = 0.565
μ_s = y.value.std() * 2

with pm.Model() as model:
    group1_mean = pm.Normal("group1_mean", mu=μ_m, sigma=μ_s)
    group2_mean = pm.Normal("group2_mean", mu=μ_m, sigma=μ_s)

In [ ]:
# priors
mean_prior, std_prior = df_returns.mean(), df_returns.std()
std_low, std_high = std_prior / 1000, std_prior * 1000
T = YEAR ** .5
mean, std, returns = {}, {}, {}
groups = df_returns.index.get_level_values(lvl1).unique()
num_groups = len(groups) # flag to plot comparisio if comparing two hists

with pm.Model() as model:
    nu = pm.Exponential('nu_minus_two', 1 / 29, testval=4) + 2.
    for i, g in enumerate(groups):
        df_g = df_returns.loc[midx[g]]
        mean[i] = pm.Normal(f'mean_{g}', mu=mean_prior, sd=std_prior, testval=df_g.mean())
        std[i] = pm.Uniform(f'std_{g}', lower=std_low, upper=std_high, testval=df_g.std())
        returns[i] = pm.StudentT(f'returns_{g}', nu=nu, mu=mean[i], sd=std[i], observed=df_g)
        pm.Deterministic(f'vol_{g}', returns[i].distribution.sd * T)
        pm.Deterministic(f'sharpe_{g}', returns[i].distribution.mean / returns[i].distribution.sd * T)
    if num_groups == 2:
        mean_diff = pm.Deterministic('mean diff', mean[0] - mean[1])
        pm.Deterministic('std diff', std[0] - std[1])
        pm.Deterministic('effect size', mean_diff / (std[0] ** 2 + std[1] ** 2) ** .5 / 2)

if inspect_mode:
    pm.model_to_graphviz(model=model)

with model:
    # HMC NUTS Sampling
    #cores, _ = print_machine_type()
    trace = pm.sample(draws=draws, tune=tune,
                      chains=chains, 
                      #cores=cores,
                      target_accept=target_accept,
                      #return_inferencedata=False, # TODO: what's for?
                      progressbar=True)